In [2]:
from pyspark.sql import SparkSession


In [3]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_sql')
 .getOrCreate()
)

In [5]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-10.csv')
)

In [6]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [8]:
"Há a necessidade de utilizar a função do spark createOrReplaceTempView para criar uma view/dataframe e ser manipulado com sql"

'Há a necessidade de utilizar a função do spark createOrReplaceTempView para criar uma view/dataframe e ser manipulado com sql'

In [9]:
df.createOrReplaceTempView('combustiveis')

In [15]:
spark.sql("""
    select * from combustiveis 
""").show(5)

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------+---------+--------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|Complemento|        Bairro|      Cep| Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------+---------+--------+--------------+--------------+---------------+-----------------+--------+
|            SE|            SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|       NULL|JARDIM ZULMIRA|18061-000|GASOLINA|    01/10/2024|          5,39|           NULL|       R$ / litro|  BRANCA|
|            SE|            SP| SOROCABA|COMPETRO COMERCIO...| 0

In [14]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
""").show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [18]:
"regexp_replace funciona como o replace do sql"

'regexp_replace funciona como o replace do sql'

In [19]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [21]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

+--------------+--------+--------------+-----------------+
|Estado - Sigla| Produto|Valor de Venda|Unidade de Medida|
+--------------+--------+--------------+-----------------+
|            SP|GASOLINA|          5.39|       R$ / litro|
|            SP|  ETANOL|          3.49|       R$ / litro|
|            BA|GASOLINA|          6.02|       R$ / litro|
|            BA|  ETANOL|          4.69|       R$ / litro|
|            BA|GASOLINA|          5.95|       R$ / litro|
+--------------+--------+--------------+-----------------+
only showing top 5 rows



In [22]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
   from view_precos
   group by all
   order by diferenca DESC
""")

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.27|       8.89| 3.620000000000001|
|            SP|          GASOLINA|       R$ / litro|       4.89|       7.99|3.1000000000000005|
|            SP|            ETANOL|       R$ / litro|       3.09|       5.99|2.9000000000000004|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.49|       7.99|               2.5|
|            RJ|          GASOLINA|       R$ / litro|       5.39|       7.79|2.4000000000000004|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows



In [23]:
df_diferenca_precos.createOrReplaceTempView('df_diferenca_precos')
df_diferenca_precos.show(5)

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.27|       8.89| 3.620000000000001|
|            SP|          GASOLINA|       R$ / litro|       4.89|       7.99|3.1000000000000005|
|            SP|            ETANOL|       R$ / litro|       3.09|       5.99|2.9000000000000004|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.49|       7.99|               2.5|
|            RJ|          GASOLINA|       R$ / litro|       5.39|       7.79|2.4000000000000004|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows

